## Attention

- 어텐션(attention)은 입력에 대한 벡터 변환을 인코더(encoder)에서 처리하고 모든 벡터를 디코더(decoder)로 보냄.
이렇게 모든 벡터를 전달하는 것은 시간이 흐를수록 초기 정보를 잃어버리는 `기울기 소실`문제를 해결하기 위함임. 
그러나 모든 벡터가 전달됨으로써 행렬 크기가 매우 커지는 단점이 있는데, 이를 해결하기 위해 `소프트맥스 함수`를 사용하여
가중합을 구하고, 그 값을 디코더에 전달함.

- 가중합이 전달되면서, 정보를 많이 전달받은 디코더에게 부담이 가기 때문에, 디코더는 은닉 상태에 대하여
중점적으로 `집중(attention)`해 보아야 할 벡터를 소프트맥스 함수로 점수매긴 후 각각을 은닉 상태 벡터들과 곱함.
그리고 이 은닉 상태를 모두 더하여 하나의 값으로 만듦. 즉, 어텐션은 모든 벡터 중 꼭 살펴봐야 할 벡터에 집중하겠다는 의미임.

## Transformer
- 트랜스포머는 어텐션을 극대화하는 방법으로, 인코더와 디코더를 여러개 중첩시킨 구조임.
이 때 각각의 인코더와 디코더를 `block`이라고 함(논문에서는 인코더와 디코더 블록을 6개씩 중첩한 구조 사용)

- 하나의 인코더는 `self-attention`과 `전방향 신경망(feed-forward neural network)`으로 구성되어 있음.
인코더에서는 단어를 벡터로 임베딩하며, 이를 셀프 어텐션과 전방향 신경망으로 전달함.
이 때 셀프 어텐션은 문장에서 각 단어끼리 얼마나 관계하는지를 계산해서 반영함. 즉, 셀프 어텐션으로
문장 안에서 단어 간 관계를 파악할 수 있음. 셀프 어텐션에서 파악된 단어간 관계는 전방향 신경망으로 전달됨

- 디코더는 층을 총 3개 가지는데, 인코더에서 넘어온 벡터가 처음 만나는 것이 self-attention 층임.(인코더와 동일)
셀프 어텐션 층을 지나면 인코더-디코더 어텐션 층이 있음. 이 층에서는 인코더가 처리한 정보를 받아 어텐션 메커니즘을 수행하고,
마지막으로 전방향 신경망으로 데이터가 전달됨.

### 어텐션 메커니즘
- 어텐션 메커니즘을 이용하기 위해서는 가장 먼저 `어텐션 스코어`를 구해야 함
- 어텐션 스코어란, 현 디코더의 시점 i에서 단어를 예측하기 위해, 인코더의 모든 은닉상태 값($h_j$)이
디코더의 현 시점 은닉 상태($s_i$)와 얼마나 유사한지(관련이 있는지)를 판단하는 값임. 따라서,
어텐션 스코어는 앞 수식처럼 인코더의 모든 은닉 상태 값($h_j$)과 디코더에서 이전 시점 은닉상태($s_{i-1}$)
값을 이용하여 구할 수 있음
- 어텐션 스코어가 계산되면, 이 값을 소프트맥스 함수에 적용하여 확률로 변환하고,
이렇게 계산된 0 ~ 1 사이의 값들이 특정 시점(timestep)에 대한 가중치, 즉 시간의 가중치가 됨.
- 시간의 가중치($a_{ij}$)와 은닉 상태($h_j$)의 가중합을 계산하면 하나의 벡터가 계산되는데, 이것이 컨텍스트 벡터(context vector)임.
- 마지막으로 디코더의 은닉 상태를 구하는데, 이를 위해 컨텍스트 벡터와 디코더 이전 시점의 은닉 상태와 출력이 필요함.

## Seq2seq
- `seq2seq(sequence to seqeunce)`는 입력 시퀀스에 대한 출력 시퀀스를 만들기 위한 모델

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os

import re
import random

device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# 데이터 정규화
def normalizeString(df, lang) : 
    sentence = df[lang].str.lower() # 소문자 전환
    sentence = sentence.str.replace('[^A-Za-z\s]+', ' ')
    sentence = sentence.str.normalize('NFD') # 유니코드 정규화
    sentence = sentence.str.encode('ascii', errors='ignore').str.decode('utf-8')
    return sentence

def read_sentence(df, lang1, lang2) : 
    sentence1 = normalizeString(df, lang1) # 데이터셋 1번째 열 
    sentence2 = normalizeString(df, lang2)
    return sentence1, sentence2

def read_file(loc, lang1, lang2) :
    df = pd.read_csv(loc, delimiter='\t', header=None, names=[lang1, lang2])
    return df

def process_data(lang1, lang2) :
    df = read_file('../080289-main/chap10/data/%s-%s.txt'%(lang1, lang2), lang1, lang2) # load data
    sentence1, sentence2 = read_sentence(df, lang1, lang2)
    
    input_lang = Lang()
    output_lang = Lang()
    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]]
            input_lang.addSentence(sentence1[i])
            output_lang.addSentence(sentence2[i])
            pairs.append(full)

    return input_lang, output_lang, pairs

In [4]:
# Tensor로 변환
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [5]:
# 인코더 네트워크
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()       
        self.input_dim = input_dim
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
              
    def forward(self, src):      
        embedded = self.embedding(src).view(1,1,-1)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

In [6]:
# 디코더 네트워크 
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
      
    def forward(self, input, hidden):
        input = input.view(1, -1)
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)       
        prediction = self.softmax(self.out(output[0]))      
        return prediction, hidden 

In [7]:
# Seq2seq 네트워크
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()
        # 인코더와 디코더 초기화
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
     
    def forward(self, input_lang, output_lang, teacher_forcing_ratio=0.5):

        input_length = input_lang.size(0) # 입력 문장 길이(문장 단어수)
        batch_size = output_lang.shape[1] 
        target_length = output_lang.shape[0]
        vocab_size = self.decoder.output_dim      
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

        for i in range(input_length):
            # 문장의 모든 단어 인코딩
            encoder_output, encoder_hidden = self.encoder(input_lang[i])
            
        # 인코더 은닉층 -> 디코더 은닉층
        decoder_hidden = encoder_hidden.to(device)  
        # 예측 단어 앞에 SOS token 추가
        decoder_input = torch.tensor([SOS_token], device=device)  

        for t in range(target_length):   
            # 현재 단어에서 출력단어 예측
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            # teacher force 활성화하면 모표를 다음 입력으로 사용
            input = (output_lang[t] if teacher_force else topi)
            # teacher force 활성화하지 않으면 자체 예측 값을 다음 입력으로 사용
            if (teacher_force == False and input.item() == EOS_token) :
                break
        return outputs
    
# teacher_force : seq2seq에서 많이 사용되는 기법. 번역(예측)하려는 모표 단어를 디코더의 다음 입력으로 넣어줌

In [8]:
# 모델의 오차 계산 함수 정의
teacher_forcing_ratio = 0.5

def Model(model, input_tensor, target_tensor, model_optimizer, criterion):
    model_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0
    output = model(input_tensor, target_tensor)
    num_iter = output.size(0)

    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot])

    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter
    return epoch_loss

In [9]:
# 모델 훈련함수 정의
def trainModel(model, input_lang, output_lang, pairs, num_iteration=20000):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()
    total_loss_iterations = 0

    training_pairs = [tensorsFromPair(input_lang, output_lang, random.choice(pairs))
                      for i in range(num_iteration)]
  
    for iter in range(1, num_iteration+1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = Model(model, input_tensor, target_tensor, optimizer, criterion)
        total_loss_iterations += loss

        if iter % 5000 == 0:
            average_loss= total_loss_iterations / 5000
            total_loss_iterations = 0
            print('%d %.4f' % (iter, average_loss))
          
    torch.save(model.state_dict(), './mytraining.pt')
    return model

In [10]:
# 모델 평가
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])  
        decoded_words = []  
        output = model(input_tensor, output_tensor)
  
        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)

            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words

def evaluateRandomly(model, input_lang, output_lang, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('input {}'.format(pair[0]))
        print('output {}'.format(pair[1]))
        output_words = evaluate(model, input_lang, output_lang, pair)
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))

In [11]:
lang1 = 'eng'
lang2 = 'fra'
input_lang, output_lang, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

input_size = input_lang.n_words
output_size = output_lang.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 75000

encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)
 
print(encoder)
print(decoder)

model = trainModel(model, input_lang, output_lang, pairs, num_iteration)

C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


random sentence ['great weather  isn t it ', 'temps magnifique  n est ce pas ']
Input : 12671 Output : 18454
Encoder(
  (embedding): Embedding(12671, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(18454, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=18454, bias=True)
  (softmax): LogSoftmax(dim=1)
)
5000 4.8856
10000 4.8447
15000 4.8432
20000 4.8344
25000 4.8230
30000 4.8251
35000 4.8005
40000 4.8146
45000 4.8014
50000 4.7815
55000 4.8139
60000 4.8271
65000 4.8388
70000 4.8086
75000 4.8200


In [12]:
evaluateRandomly(model, input_lang, output_lang, pairs)

input have you ever eaten anything that made you hallucinate 
output as tu jamais mang  quoi que ce soit qui t ait fait halluciner  
predicted je est a pas       <EOS>
input do you want to go to the station with me 
output voulez vous venir  avec moi  jusqu  la gare  
predicted je est a pas       <EOS>
input i m surprised you didn t know that 
output je suis surpris que tu ignorais cela 
predicted je est a pas     
input he asked me for help 
output il m a demand  de l aide 
predicted je est a pas      
input i m not afraid of you 
output je n ai pas peur de vous 
predicted je est a pas     
input how would you like us to proceed 
output comment voudrais tu que nous proc dions  
predicted je est a pas      
input my father quit drinking 
output mon p re a arr t  de boire 
predicted je est a pas       <EOS>
input i hate this town 
output je d teste cette ville 
predicted je est a pas   
input he is worthy of our praise 
output il m rite nos louanges 
predicted je est a pas   
input don 